## Object detection and Threshold/Occupancy maker..

### Object detection

In [1]:
import torch
from pathlib import Path
import cv2
from google.colab import drive
import numpy as np


model_path = 'best.pt'  # Model

# Check if the model file exists
if not Path(model_path).exists():
    raise FileNotFoundError(f"Model file not found at {model_path}")

# Load the model
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)

# Load an image
img_path = 'saved_images/output/stitched_image_output.png'
if not Path(img_path).exists():
    raise FileNotFoundError(f"Image file not found at {img_path}")

# Perform inference / Prediction
results = model(img_path)


def render_without_labels(results):
    for i, (im, pred) in enumerate(zip(results.ims, results.pred)): # Use results.ims instead of results.imgs
        if pred is not None:
            for *box, conf, cls in reversed(pred):  # xyxy, confidence, class
                label = f'{results.names[int(cls)]} {conf:.2f}' if conf > 0 else f'{results.names[int(cls)]}'
                # Remove label and confidence display
                # Make below comment line executable to get object namings
                # cv2.putText(im, label, (int(box[0]), int(box[1]) - 2), 0, 0.6, [225, 255, 255], thickness=1, lineType=cv2.LINE_AA)
                cv2.rectangle(im, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (255, 0, 0), -1)
        yield im

displayed_image_np = list(render_without_labels(results))

# Save the results
for i, img_np in enumerate(displayed_image_np):
    output_image_path = f'output_image_{i}.png'
    cv2.imwrite(output_image_path, img_np)
    print(f"Displayed image saved locally as {output_image_path}")

/usr/local/lib/python3.10/dist-packages/torch/hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 185.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 175.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 8.9s, installed 3 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-7-13 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
WARNING ⚠️ NMS time limit 0.550s exceeded


Displayed image saved locally as output_image_0.png


### Normal png creator

In [2]:
## Marked
import cv2
import numpy as np

# Function to apply thresholding to an image
def apply_threshold(image, threshold_value):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresholded_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)
    return thresholded_image


# Define threshold value (adjust as needed)
threshold_value = 150

# Apply thresholding to each displayed image and save
for i, img_np in enumerate(displayed_image_np):
    thresholded_img = apply_threshold(img_np, threshold_value)
    output_image_path = f'output_image_{i}_thresholded.png'
    cv2.imwrite(output_image_path, thresholded_img)
    print(f"Thresholded image saved locally as {output_image_path}")

Thresholded image saved locally as output_image_0_thresholded.png


### Occcupancy grid maker with 512x512 resolution and with .pgm extension

In [3]:
import cv2
import numpy as np
import time

# Function to apply thresholding to an image
def apply_threshold(image, threshold_value):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresholded_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)
    return thresholded_image

# Function to resize an image to 512x512 resolution
def resize_image(image, width, height):
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

# Assuming you have `displayed_image_np` from your previous code
# displayed_image_np = []  # Placeholder for your list of images

# Define threshold value (adjust as needed)
threshold_value = 150

# Desired output resolution
output_width = 512
output_height = 512

# Apply thresholding, resize and save each displayed image
for i, img_np in enumerate(displayed_image_np):
    # Start timing
    start_time = time.time()

    # Apply thresholding
    thresholded_img = apply_threshold(img_np, threshold_value)

    # Resize to 512x512
    resized_img = resize_image(thresholded_img, output_width, output_height)

    # Save as .pgm file
    output_image_path = f'output_image_{i}_thresholded.pgm'
    cv2.imwrite(output_image_path, resized_img)

    # End timing
    end_time = time.time()

    # Calculate elapsed time
    elapsed_time = end_time - start_time
    print(f"Thresholded image saved locally as {output_image_path} in {elapsed_time:.4f} seconds")

print("All images have been processed and saved.")



Thresholded image saved locally as output_image_0_thresholded.pgm in 0.0079 seconds
All images have been processed and saved.
